# Installation

In [1]:
!pip install flask_ngrok

In [2]:
!pip install pyngrok

     |████████████████████████████████| 745 kB 5.4 MB/s 
  Created wheel for pyngrok: filename=pyngrok-5.1.0-py3-none-any.whl size=19007 sha256=5a00794785d429f8ae17a6273dc635dda56f5ce4843d8fed0031e09ae5e3b844
  Stored in directory: /root/.cache/pip/wheels/bf/e6/af/ccf6598ecefecd44104069371795cb9b3afbcd16987f6ccfb3
Successfully built pyngrok


# Flask App Source

In [14]:
%cd /content/drive/MyDrive/AI/TemperatureForecast/App

/content/drive/MyDrive/AI/TemperatureForecast/App


In [15]:
%%writefile src/datacontroller/modeldatabase.py
import numpy as np
import pandas as pd
import datetime

class ModelDatabase:
    def __init__(self,filename):
        self.features = ['temp', 'day_cos', 'day_sin', 'month_sin', 'month_cos', 'humidity']        
        df = pd.read_csv(filename)
        self.hist_data = self.dataframe_to_array_features(df,self.features)
        self.times = [x for x in df["time"]]
        self.MAX_SIZE_HIST_DATA = 200

    def dataframe_to_array_features(self,df:pd.DataFrame,features:list)->np.array:
        # Converting the dt column to datetime object 
        df['time'] = [datetime.datetime.strptime(x, '%Y-%m-%d %H:%M:%S') for x in df['time']]

        # Sorting by the date 
        df.sort_values('time', inplace=True)
        # Extracting the hour of day
        df['hour'] = [x.hour+x.minute/60 for x in df['time']]

        # Creating the cyclical daily feature 
        df['day_cos'] = [np.cos(x * (2 * np.pi / 24)) for x in df['hour']]
        df['day_sin'] = [np.sin(x * (2 * np.pi / 24)) for x in df['hour']]

        # Extracting the timestamp from the datetime object 
        df['timestamp'] = [x.timestamp() for x in df['time']]

        # Seconds in day 
        s = 24 * 60 * 60 

        # Seconds in year 
        year = (365.25) * s

        df['month_cos'] = [np.cos((x) * (2 * np.pi / year)) for x in df['timestamp']]
        df['month_sin'] = [np.sin((x) * (2 * np.pi / year)) for x in df['timestamp']]
        # print(df[features_final].tail(48))
        return np.array(df[features])

    def handle_dict_data(self,new_data:dict)->np.array:
        df = pd.DataFrame.from_dict(new_data)
        return self.dataframe_to_array_features(df,self.features)

    def add_data(self,dict_data:dict):
        self.times.append(datetime.datetime.strptime(dict_data["time"], '%Y-%m-%d %H:%M:%S'))
        array_data = self.handle_dict_data(dict_data)
        self.hist_data = np.vstack([self.hist_data,array_data])
        n_rows, n_cols = self.hist_data.shape
        if n_rows>self.MAX_SIZE_HIST_DATA:
            self.hist_data = self.hist_data[-self.MAX_SIZE_HIST_DATA:]
            self.times = self.times[-self.MAX_SIZE_HIST_DATA:]

    def get_input_for_model(self,lag=48)->np.array:
        n_features = self.hist_data.shape[1]
        
        # Creating placeholder lists
        X= []
        X.append(self.hist_data[-lag:])

        X= np.array(X)

        # Reshaping the X array to an RNN input shape 
        X = np.reshape(X, (X.shape[0], lag, n_features))

        return X

    def get_time_for_predicted_val(self,n_ahead):
        predicted_times = []
        latest_time = self.times[-1]
        for i in range(n_ahead):
            predicted_times.append(str(latest_time+datetime.timedelta(minutes = 30*(i+1))))
        return predicted_times

Overwriting src/datacontroller/modeldatabase.py


In [16]:
%%writefile src/models/model.py
from abc import ABC, abstractmethod
import numpy as np
from tensorflow import keras
from keras.models import Model
from datacontroller.modeldatabase import ModelDatabase

class AbstractModel:
    def __init__(self,database:ModelDatabase):
        self.model = self.load_model()
        self.database = database

    @abstractmethod
    def load_model(self)->Model:
        pass

    def predict(self)->np.array:
        input = self.create_inp_predict()
        return self.model.predict(input)

    @abstractmethod
    def create_inp_predict(self)->np.array:
        pass


Overwriting src/models/model.py


In [17]:
%%writefile src/models/temperaturemodels/SingleStepTemperatureModel.py
from models.model import AbstractModel
from datacontroller.modeldatabase import ModelDatabase

import numpy as np
from tensorflow import keras
from keras.models import Input, Model, Sequential


class SingleStepTemperatureModel(AbstractModel):
    def __init__(self,database:ModelDatabase):
        super().__init__(database)
        self.lag = 48

    def load_model(self)->Model:
        return keras.models.load_model("models/singlesteptemperatureforecastmodel")
    
    # def create_inp_predict(self)->np.array:
    #     n_features = self.hist_data.shape[1]
    #     inp = []
    #     inp.append(self.hist_data[-self.lag:])
    #     inp = np.array(inp)
    #     inp = np.reshape(inp,(inp.shape[0],self.lag,n_features))
    #     return inp
    def create_inp_predict(self)->np.array:
        return self.database.get_input_for_model(self.lag)


Overwriting src/models/temperaturemodels/SingleStepTemperatureModel.py


In [18]:
%%writefile src/models/temperaturemodels/TemperatureModel.py
from models.model import AbstractModel
from datacontroller.modeldatabase import ModelDatabase

import numpy as np
from tensorflow import keras
from keras.models import Input, Model, Sequential

class TemperatureModel(AbstractModel):
    def __init__(self,database:ModelDatabase):
        super().__init__(database)
        self.lag = 144

    def load_model(self)->Model:
        return keras.models.load_model("models/temperatureforecastmodel")
    
    # def create_inp_predict(self)->np.array:
    #     n_features = self.hist_data.shape[1]
    #     inp = []
    #     inp.append(self.hist_data[-self.lag:])
    #     inp = np.array(inp)
    #     inp = np.reshape(inp,(inp.shape[0],self.lag,n_features))
    #     return inp

    def create_inp_predict(self)->np.array:
        return self.database.get_input_for_model(self.lag)


Overwriting src/models/temperaturemodels/TemperatureModel.py


In [19]:
%%writefile src/models/humiditymodels/SingleStepHumidityModel.py
from models.model import AbstractModel
from datacontroller.modeldatabase import ModelDatabase

import numpy as np
from tensorflow import keras
from keras.models import Input, Model, Sequential

class SingleStepHumidityModel(AbstractModel):
    def __init__(self,database:ModelDatabase):
        super().__init__(database)
        self.lag = 48

    def load_model(self)->Model:
        return keras.models.load_model("models/singlestephumidityforecastmodel")
    
    # def create_inp_predict(self)->np.array:
    #     n_features = self.hist_data.shape[1]
    #     inp = []
    #     inp.append(self.hist_data[-self.lag:])
    #     inp = np.array(inp)
    #     inp = np.reshape(inp,(inp.shape[0],self.lag,n_features))
    #     return inp

    def create_inp_predict(self)->np.array:
        return self.database.get_input_for_model(self.lag)
    

    # def create_hist_data(self)->np.array:
    #     features_final = ['temp', 'day_cos', 'day_sin', 'month_sin', 'month_cos', 'humidity']        
    #     return datahandler.csv_to_array_features("mockdata/mockdata.csv",features_final)

Overwriting src/models/humiditymodels/SingleStepHumidityModel.py


In [20]:
%%writefile src/models/humiditymodels/HumidityModel.py
from models.model import AbstractModel
from datacontroller.modeldatabase import ModelDatabase

import numpy as np
from tensorflow import keras
from keras.models import Input, Model, Sequential

class HumidityModel(AbstractModel):
    def __init__(self,database:ModelDatabase):
        super().__init__(database)
        self.lag = 144

    def load_model(self)->Model:
        return keras.models.load_model("models/humidityforecastmodel")
    
    # def create_inp_predict(self)->np.array:
    #     n_features = self.hist_data.shape[1]
    #     inp = []
    #     inp.append(self.hist_data[-self.lag:])
    #     inp = np.array(inp)
    #     inp = np.reshape(inp,(inp.shape[0],self.lag,n_features))
    #     return inp
    def create_inp_predict(self)->np.array:
        return self.database.get_input_for_model(self.lag)

    # def create_hist_data(self)->np.array:
    #     features_final = ['temp', 'day_cos', 'day_sin', 'month_sin', 'month_cos', 'humidity']        
    #     return datahandler.csv_to_array_features("mockdata/mockdata.csv",features_final)

Overwriting src/models/humiditymodels/HumidityModel.py


In [36]:
%%writefile src/main.py
from datacontroller.modeldatabase import ModelDatabase
from models.temperaturemodels.TemperatureModel import TemperatureModel
from models.temperaturemodels.SingleStepTemperatureModel import SingleStepTemperatureModel
from models.humiditymodels.HumidityModel import HumidityModel
from models.humiditymodels.SingleStepHumidityModel import SingleStepHumidityModel

import numpy as np
import json

from flask import Flask, request, redirect, url_for
from flask_ngrok import run_with_ngrok
from pyngrok import ngrok
app = Flask(__name__)
run_with_ngrok(app)   
  
@app.route("/")
def home():
    return "<h1>Air Quality Prediction</h1>"
    
['temp', 'day_cos', 'day_sin', 'month_sin', 'month_cos', 'humidity']            
def html_table(table:np.array,times: list):
    yield '<html>\
            <head>\
            <style>\
            table, th, td {\
              border: 1px solid black;\
              border-collapse: collapse;\
            }\
            </style>\
            </head>\
            <body>\
              <table>\
                <tr>\
                  <th>Time</th>\
                  <th>Temperature</th> \
                  <th>DayCos</th>\
                  <th>DaySin</th>\
                  <th>MonthSin</th>\
                  <th>MonthCos</th>\
                  <th>Humidity</th>\
                </tr>'
    for time,row in zip(times,table):
      yield '  <tr><td>'
      yield '    </td><td>'.join(map(str,[time]))
      yield '    </td><td>'
      yield '    </td><td>'.join(map(str,row.tolist()))
      yield '  </td></tr>'
    yield '</table>\
          </body>\
        </html>'    

@app.route("/history")
def get_hist_data():
    table_data = model_database.hist_data
    times = model_database.times
    return "".join(html_table(table_data,times))

@app.route("/publish",methods = ["POST"])
def update_hist_data():
    json_data = request.get_json()
    # dict_data=json.loads(json_data)
    model_database.add_data(json_data)
    return redirect(url_for('get_hist_data'))

@app.route("/multistep")
def multi_step_predict():
    n_ahead = 24
    predicted_times = model_database.get_time_for_predicted_val(n_ahead)

    data = {}
    data["predictions"]=[]

    temp_list = temperature_model.predict()[0]
    humidity_list = humidity_model.predict()[0]

    for i in range(n_ahead):
        data["predictions"].append({"time":predicted_times[i],
                                    "temperature":str(temp_list[i]),
                                    "humidity":str(humidity_list[i])})
    print(data)
    return data

@app.route("/singlestep")
def single_step_predict():
    data = {}
    data["predictions"]=[]
    data["predictions"].append({"time":model_database.get_time_for_predicted_val(1)[0],
                                "temperature":str(single_step_temperature_model.predict()[0][0]),
                                "humidity":str(single_step_humidity_model.predict()[0][0])})
    print(data)
    return data

model_database = ModelDatabase("mockdata/mockdata.csv")

temperature_model = TemperatureModel(model_database)
single_step_temperature_model = SingleStepTemperatureModel(model_database)
humidity_model = HumidityModel(model_database)
single_step_humidity_model = SingleStepHumidityModel(model_database)

def main():
    app.run()
    # mul_temp = temperature_model.predict()
    # sing_temp = single_step_temperature_model.predict()
    # mul_hum = humidity_model.predict()
    # sing_hum = single_step_humidity_model.predict()
    # print("Multi Temp")
    # print(mul_temp)
    # print("==========")
    # print("Sing Temp")
    # print(sing_temp)
    # print("==========")
    # print("Multi Hum")
    # print(mul_hum)
    # print("==========")
    # print("Sing Hum")
    # print(sing_hum)
    # print("==========")


if __name__ == "__main__":
    main()
    

Overwriting src/main.py


# Demo Deploy

In [37]:
# !ngrok authtoken 24lZUtBho84hgbsKXh9lLQDXwTr_4yhicnAqGTnkqcCRsRgdT

In [42]:
# !ls
!python3 src/main.py

2022-02-08 04:43:18.778504: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
 * Serving Flask app "main" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off
 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
 * Running on http://a25b-34-71-178-119.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040
127.0.0.1 - - [08/Feb/2022 04:43:27] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [08/Feb/2022 04:43:27] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [08/Feb/2022 04:43:29] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [08/Feb/2022 04:43:31] "GET /history HTTP/1.1" 200 -
127.0.0.1 - - [08/Feb/2022 04:43:32] "GET /favicon.ico HTTP/1.1" 404 -
Dict Data:
{'time': ['2022-01-01 00:00:00'], 'temp': <map object at 0x7fb573d4b890>, 'humidity': <map object at 0x7fb573d4b810>}
Handling
127.0.0.1 - - [08/Feb/2022 04:43:49] "POST /publish HTTP/1.1" 302 -
127.0.0.1 - 

In [ ]:
# features_final = ['temp', 'day_cos', 'day_sin', 'month_sin', 'month_cos', 'humidity']
# arr = csv_to_array_features("mockdata/mockdata.csv",features_final)
# # # %cd ..
# # # !ls
# single_step_temperature_model = SingleStepTemperatureModel()
import datetime

mytime = datetime.datetime.strptime("2022-01-02 11:30:00", '%Y-%m-%d %H:%M:%S')
next_30 = mytime+datetime.timedelta(minutes=90)
str(next_30)
# curl http://a25b-34-71-178-119.ngrok.io/publish -d '{"time": "2022-01-01 00:00:00","temp":12, "humidity":76}' -H 'Content-Type: application/json'

'2022-01-02 13:00:00'

In [ ]:
# res =  single_step_temperature_model.predict()
import numpy as np
a = np.array([[1,2,3]])
b= a[0]
tmp = {}
tmp["prediction"] = []
for i in range(3):
    tmp["prediction"].append({"idx":i,"val":b[i]})  
tmp

{'prediction': [{'idx': 0, 'val': 1},
  {'idx': 1, 'val': 2},
  {'idx': 2, 'val': 3}]}

In [30]:
# df = pd.read_csv("mockdata/mockdata.csv")
# df['time'] = [datetime.datetime.strptime(x, '%Y-%m-%d %H:%M:%S') for x in df['time']]

# # Sorting by the date 
# df.sort_values('time', inplace=True)
# # Extracting the hour of day
# df['hour'] = [x.hour+x.minute/60 for x in df['time']]
# df['hour'].tail(48)
# import numpy as np
a = np.array([[1,2,3],[12,23,4]])
# b= [1,2]
# for el1, el2 in zip(a,b):
#     print(el1)
#     print(el2)
#     print("=====")
b= a[0]
b=map(str,b.tolist())
st = "test ".join(b)
st


'1test 2test 3'

In [ ]:
import pandas as pd
df = pd.read_csv("data/final_weather0721.csv")

In [ ]:
# df.head()
data={}
data["temp"]= 50
name ="Long"
# print(data["temp"])

50


In [ ]:
last_200_df = df.tail(200)

In [ ]:
# last_200_df.to_csv("mockdata/mockdata.csv",index=False)
# last_200_df = pd.read_csv("mockdata/mockdata.csv")
# last_200_df.head()
# last_200_arr = np.array(last_200_df)
last_200_arr